In [1]:
import collections
import re
from d2l import torch as d2l
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

/home/liangtian/anaconda3/envs/d2l/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/liangtian/anaconda3/envs/d2l/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [4]:
# 划分词元
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines,)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [11]:
def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)
counter = count_corpus(tokens)


print(counter)
print(counter.items())
# 对元组组成的列表按某个元素排序
print(sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True))


Counter({'the': 2261, 'i': 1267, 'and': 1245, 'of': 1155, 'a': 816, 'to': 695, 'was': 552, 'in': 541, 'that': 443, 'my': 440, 'it': 437, 'had': 354, 'me': 281, 'as': 270, 'at': 243, 'for': 221, 'with': 216, 'but': 204, 'time': 200, 'were': 158, 'this': 152, 'you': 137, 'on': 137, 'then': 134, 'his': 129, 'there': 127, 'he': 123, 'have': 122, 'they': 122, 'from': 122, 'one': 120, 'all': 118, 'not': 114, 'into': 114, 'upon': 113, 'little': 113, 'so': 112, 'is': 106, 'came': 105, 'by': 103, 'some': 94, 'be': 93, 'no': 92, 'could': 92, 'their': 91, 'said': 89, 'saw': 88, 'down': 87, 'them': 86, 'machine': 85, 'which': 85, 'very': 85, 'or': 84, 'an': 84, 'we': 82, 'now': 79, 'what': 77, 'been': 75, 'these': 74, 'like': 74, 'her': 74, 'out': 73, 'seemed': 72, 'up': 71, 'man': 70, 'about': 70, 's': 70, 'its': 69, 'thing': 66, 'again': 62, 'traveller': 61, 'would': 60, 'more': 59, 'white': 59, 'our': 57, 'thought': 57, 'felt': 57, 'when': 55, 'over': 54, 'weena': 54, 'still': 53, 'world': 52, 

In [36]:


# 构建词表
class Vocab:
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        if tokens is None:
            tokens=[]
        if reserved_tokens is None:
            reserved_tokens=[]
        
        self.id_to_token=['<unk>'] + reserved_tokens
        self.token_to_id={tokens:id for id,tokens in enumerate(self.id_to_token)}
        counter=count_corpus(tokens)
        self._token_freqs=sorted(counter.items(),key=lambda x:x[1],reverse=True)
        # 遍历字典的键值对，学到了，调用items方法
        for token,freq in self._token_freqs:
            if freq<min_freq:
                break
            else:
                self.id_to_token.append(token)
                self.token_to_id[token]=len(self.id_to_token)-1


    def __len__(self):
        return len(self.id_to_token)


    def __getitem__(self,tokens):
        '''
        get 方法是字典的一种安全访问方式。它尝试获取 tokens 对应的值，如果 tokens 不存在于字典中，
        则返回默认值 self.unk（通常是未知词元的索引，0）。
        这种方式确保了即使 tokens 不存在于 token_to_idx 字典中，代码也不会抛出异常，而是返回一个预定义的默认值。
        '''
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_id.get(tokens,self.token_to_id['<unk>'])
        
        # 自己调用自己，递归
        return [self.__getitem__(token) for token in tokens]



    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.id_to_token[indices]
        return [self.id_to_token[index] for index in indices]
    


    @property
    # Python 中的 属性方法（@property），它们使得类的某些方法看起来像属性一样，可以通过点（.）符号进行访问，
    # 而不需要加上括号 ()。
    # 使用 @property 装饰器的属性可以在访问时计算，而不是在对象创建时计算。这对于某些计算开销较大的属性来说，可以提高性能。
    # @property 可以创建只读属性，不允许外部代码修改该属性的值。这样可以更好地保护类的内部状态。
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

In [43]:
# 尝试调用
# dict.items()返回一个视图对象，而不是一个可以通过索引访问的列表。视图对象不支持切片操作
vocab = Vocab(tokens)
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])


文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [52]:
# 整合
def load_array(max_tokens=-1):
    lines=read_time_machine()
    tokens=tokenize(lines)
    vocab=Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    # 提供一个max_tokens参数来限制载入的词元数量
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus,vocab


corpus,vocab=load_array()
print(len(vocab))
print(corpus)
print(len(corpus))
print(vocab.token_freqs)

4580
[1, 19, 50, 40, 2183, 2184, 400, 2, 1, 19, 71, 16, 37, 11, 115, 42, 680, 6, 586, 4, 108, 7, 1420, 5, 2185, 587, 6, 126, 25, 330, 127, 439, 3, 2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1, 148, 588, 825, 3, 1, 244, 2187, 4, 1, 2188, 2189, 8, 1, 2190, 4, 589, 363, 1, 2191, 9, 826, 3, 288, 8, 75, 2192, 75, 1422, 274, 25, 2193, 2194, 3, 1423, 126, 230, 131, 2195, 6, 42, 186, 35, 3, 26, 7, 9, 2196, 116, 289, 1424, 78, 76, 2197, 2198, 827, 4, 1, 2199, 4, 2200, 3, 27, 132, 11, 6, 126, 8, 21, 114, 1425, 1, 1046, 17, 5, 1426, 1427, 14, 55, 186, 3, 2201, 2202, 25, 2203, 79, 21, 153, 681, 14, 55, 76, 11, 3, 25, 2204, 22, 85, 440, 13, 590, 2, 502, 28, 6, 2205, 31, 53, 154, 1047, 9, 91, 180, 2206, 682, 1, 828, 16, 364, 29, 1428, 22, 15, 2207, 38, 2208, 23, 5, 2209, 38, 33, 9, 230, 5, 275, 69, 6, 829, 126, 6, 830, 35, 46, 235, 54, 2210, 1429, 17, 164, 591, 2, 187, 33, 503, 6, 1430, 22, 6, 2211, 683, 276, 1431, 236, 16, 11, 22, 115, 197, 831, 14, 181, 14, 2, 504, 30, 22, 22, 151, 4, 313, 9, 